An extremely slow, numerically unstable PCA implementation in TensorFlow.

In [8]:
import time
import numpy as np
from sklearn import decomposition
import tensorflow as tf

np.random.seed(5)
%matplotlib inline

In [2]:
def tflow_pca(X, n_components, iters=5000):
    """Find the first n_components principal components of array X"""
    def _norm(arr, axis=0):
        return arr / np.linalg.norm(arr, ord=2, axis=axis)
    
    #
    # Make covariance matrix to be placed in tensorflow placeholder X
    #
    orig_X = X[:].astype(np.float32)
    X_mean = np.mean(orig_X, axis=0)
    cov_X = np.cov((orig_X - X_mean).T)
    X = tf.placeholder(tf.types.float32, shape=cov_X.shape)
    
    # store the results in store_pcs
    store_pcs = [None for n in range(n_components)]
    
    #
    # Create tensorflow vars and functions
    #
    pcs = tf.Variable(tf.random_uniform([cov_X.shape[0], 1], -1.0, 1.0))
    X_s = tf.matmul(pcs, tf.transpose(pcs))
    loss = tf.reduce_mean(tf.square(X-X_s))
    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = optimizer.minimize(loss)
    
    #
    # Init tensorflow and run it
    #
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    
    for n in range(n_components):
        for step in xrange(0, iters):
            sess.run(train, feed_dict={X:cov_X})
        
        store_pcs[n] = sess.run(pcs).T
        estd_X = sess.run(X_s)
        
        assert np.dot(store_pcs[n].T, store_pcs[n]).shape == cov_X.shape
        
        store_pcs[n] = np.squeeze(_norm(store_pcs[n], axis=1))
        
        cov_X = cov_X - estd_X
    
    return store_pcs, np.dot(np.hstack(store_pcs), np.hstack(store_pcs).T)

Make some data that should be easy to PCA.

In [3]:
n_components = 3

N, M = 1000, 4
pad1 = np.random.random((N/2,M/4)); pad1.resize((N,M))
pad2 = np.random.random((N/4,M/2)); pad2.resize((N,M))
X = np.random.random((N,M)) + pad1 + pad2
X[:,1] = -X[:,0]*np.random.random(X[:,0].shape)

First run the standard [sklearn.decomposition.PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) (probabilistic PCA).

In [4]:
start_time = time.time()
pca = decomposition.PCA(n_components=n_components)
pca.fit(X)
np_pcs = pca.components_
print "Ran sklearn.decomposition.PCA in {:.3f} seconds".format(time.time() - start_time)

Ran sklearn.decomposition.PCA in 0.001 seconds


Then run my TensorFlow version.

In [5]:
start_time = time.time()
tf_pcs, _prod = tflow_pca(X, n_components=n_components, iters=5000)
print "Ran tflow_pca in {:.3f} seconds".format(time.time()-start_time)

Ran tflow_pca in 8.929 seconds


In [7]:
assert len(tf_pcs) == len(np_pcs) == n_components

for n in range(n_components):
    print np_pcs[n], tf_pcs[n], "close?", 
    print np.allclose(np_pcs[n], tf_pcs[n], rtol=1e-3) or np.allclose(np_pcs[n], -tf_pcs[n], rtol=1e-3)

[-0.57982275  0.34963527 -0.47948609 -0.55825966] [ 0.5798223  -0.349635    0.4794865   0.55825996] close? True
[ 0.56307907 -0.47916176 -0.47679008 -0.47541265] [ 0.56308001 -0.47916183 -0.47679517 -0.47540632] close? True
[-0.02672209  0.02842147 -0.73505252  0.67688695] [ 0.0267279  -0.02842641  0.73504561 -0.67689401] close? True
